In [1]:
# sentiment_analysis_model.py
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

In [2]:
# Load the IMDb dataset
df = pd.read_csv('imdb-dataset.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

# Create a pipeline with CountVectorizer and Naive Bayes classifier
model = make_pipeline(CountVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

# Print evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy}')

# Print additional analysis
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save the model to a .pkl file
joblib.dump(model, 'sentiment_analysis_model.pkl', protocol=4)

# Test the model with various inputs
example_reviews = [
    "This movie is amazing! I loved every moment of it.",
    "The plot was confusing, and the acting was not great.",
    "A masterpiece! The cinematography and acting were superb.",
    "I couldn't follow the storyline, and the characters were poorly developed."
]

print("\nPredictions for Example Reviews:")
for review in example_reviews:
    prediction = model.predict([review])[0]
    print(f"Review: '{review}'\nSentiment: {prediction}\n")


Model Accuracy: 0.8488

Classification Report:
              precision    recall  f1-score   support

    negative       0.83      0.88      0.85      4961
    positive       0.87      0.82      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000


Confusion Matrix:
[[4361  600]
 [ 912 4127]]

Predictions for Example Reviews:
Review: 'This movie is amazing! I loved every moment of it.'
Sentiment: positive

Review: 'The plot was confusing, and the acting was not great.'
Sentiment: negative

Review: 'A masterpiece! The cinematography and acting were superb.'
Sentiment: positive

Review: 'I couldn't follow the storyline, and the characters were poorly developed.'
Sentiment: negative



Time Consumption:

a. Timing Predictions: Use the time module to measure the time it takes to make predictions on the test data. This is crucial, especially if your model needs to be deployed in a real-time setting.

b. Batch Processing: If you plan to deploy this model in a production environment, consider testing its performance with a batch of inputs.

In [3]:
import time

start_time = time.time()
y_pred = model.predict(X_test)
end_time = time.time()

time_taken = end_time - start_time
print(f"Time taken for predictions: {time_taken} seconds")


Time taken for predictions: 4.743443012237549 seconds


Cross-Validation:

a. Cross-Validation Scores: Instead of a single train-test split, you might perform k-fold cross-validation to get a more robust estimate of your model's performance.

In [4]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5)  # 5-fold cross-validation
print("Cross-Validation Scores:", scores)


Cross-Validation Scores: [0.84575  0.850875 0.85675  0.840125 0.846   ]


Hyperparameter Tuning:

a. Grid Search: If applicable, you might perform a grid search to find the optimal hyperparameters for your model.

In [5]:
from sklearn.model_selection import GridSearchCV

param_grid = {'multinomialnb__alpha': [0.1, 1.0, 10.0]}
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)


Best Hyperparameters: {'multinomialnb__alpha': 0.1}


Accuracy:

Formula:
Number of Correct Predictions
____________________________________
Total Number of Predictions
​

Interpretation: The proportion of correctly classified instances.

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Precision:

Formula:
True Positives
_________________________________
True Positives + False Positives
​

Interpretation: The ability of the model to avoid labeling negative instances as positive.

In [ ]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, pos_label='positive')  # Specify positive label
print(f'Precision: {precision}')


Recall (Sensitivity):

Formula:
True Positives
_____________________________________
True Positives + False Negatives
​

Interpretation: The ability of the model to identify all relevant instances.

In [ ]:
from sklearn.metrics import recall_score

recall = recall_score(y_test, y_pred, pos_label='positive')  # Specify positive label
print(f'Recall: {recall}')


F1-Score:

Formula:
2 ×  Precision × Recall
    ____________________
      Precision + Recall

​

Interpretation: The harmonic mean of precision and recall.

In [ ]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred, pos_label='positive')  # Specify positive label
print(f'F1-Score: {f1}')


Confusion Matrix:

Interpretation: A table that shows the number of true positives, true negatives, false positives, and false negatives.

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)


Receiver Operating Characteristic (ROC) Curve and Area Under the Curve (AUC):

Interpretation: Useful for binary classification problems. The ROC curve shows the trade-off between true positive rate and false positive rate.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:, 1], pos_label='positive')
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()
